# 第6课：随机近似与随机梯度下降

> 关键词：Robbins-Monro、Stochastic Approximation、SGD、收敛性、梯度优化

---

## Part 1：通过例子介绍 Iterative Mean Estimation（均值迭代估计）

我们考虑一个基础问题：

> 如何从一个**未知分布中抽样**，估计其数学期望 $\mu = \mathbb{E}[X]$？

---

### 方法1：批量平均

若已获得 $n$ 个样本 $x_1, \dots, x_n$，用

$$
\hat{\mu}_n = \frac{1}{n} \sum_{i=1}^n x_i
$$

这就是普通的**样本均值**。

---

### 方法2：迭代均值更新

如果数据是**流式（online）**到达，能否用**迭代方式更新估计值**？

假设已有估计 $\hat{\mu}_n$，新来一个样本 $x_{n+1}$，则更新公式为：

$$
\hat{\mu}_{n+1} = \hat{\mu}_n + \alpha_n \left( x_{n+1} - \hat{\mu}_n \right)
$$

其中 $\alpha_n = \frac{1}{n+1}$，称为**学习率（step-size）**

这个思想，就是 **Robbins-Monro 随机近似算法**的雏形。







## Part 2：Robbins-Monro 算法介绍与例子

---

### 背景与问题设定

Robbins-Monro（1951）提出了一种用于解决如下 **根估计问题（root-finding problem）** 的 **随机近似算法**：

> 已知函数 $f(\theta)$ 的值不可直接获得，但可以在每次给定 $\theta$ 后获得一个带噪的观测 $Y$，其期望为 $f(\theta)$，即：
>
> $$
> \mathbb{E}[Y \mid \theta] = f(\theta)
> $$
>
> 目标是找到使 $f(\theta) = 0$ 成立的参数 $\theta^*$。

---

### Robbins-Monro 算法公式

Robbins-Monro 提出的更新规则为：

$$
\theta_{n+1} = \theta_n - \alpha_n Y_n
$$

其中：

* $Y_n$：对 $f(\theta_n)$ 的噪声观测；
* $\alpha_n$：逐步减小的学习率（step size）；
* $\theta_n$：第 $n$ 步的估计值。

目标是在观测过程中逐渐逼近 $\theta^*$，使得 $f(\theta^*) = 0$。

---

### 简单例子：求期望为零的分布均值

假设我们想估计随机变量 $X$ 的均值（即 $\mu = \mathbb{E}[X]$），但我们不知道其真实分布，只能从中采样。

转化为求解方程：

$$
f(\theta) = \mathbb{E}[X - \theta] = 0
\quad \Rightarrow \quad \theta = \mathbb{E}[X]
$$

Robbins-Monro 更新：

$$
\theta_{n+1} = \theta_n + \alpha_n (X_n - \theta_n)
$$

这就是一个 **迭代均值估计公式**，你可能已经在 MC 算法中看到类似形式。



## Part 3：Robbins-Monro 收敛性与应用

---

### 收敛性条件（Theoretical Guarantee）

Robbins 和 Monro 在论文中给出了以下收敛条件：

设：

$$
\theta_{n+1} = \theta_n - \alpha_n (f(\theta_n) + \text{noise})
$$

要保证收敛到 $\theta^*$，需要满足以下：

1. 学习率序列满足：

   $$
   \sum_{n=1}^\infty \alpha_n = \infty, \quad
   \sum_{n=1}^\infty \alpha_n^2 < \infty
   $$

   常见选择：$\alpha_n = \frac{1}{n}$

2. 函数 $f(\theta)$ 在 $\theta^*$ 附近 **连续单调**（比如线性）

3. 噪声项方差有限（有界）

---

### 实际应用

Robbins-Monro 是许多现代算法的理论基础，尤其在强化学习中：

| 应用              | 描述                                                              |
| --------------- | --------------------------------------------------------------- |
| TD(0)           | 值函数更新：$V(s) \leftarrow V(s) + \alpha (r + \gamma V(s') - V(s))$ |
| Q-learning      | 增量更新动作值：$Q(s,a) \leftarrow Q(s,a) + \alpha (TD\ error)$         |
| Policy Gradient | REINFORCE 算法中对期望梯度的估计                                           |
| Deep Q-Learning | DQN 中对目标 Q 值的估计使用类似思想                                           |
| 均值估计            | $\mu \leftarrow \mu + \alpha (x - \mu)$：在线均值计算                  |

---

### 直观理解

Robbins-Monro 的思想可以概括为：

> **每次观察都带噪声，我们就一步步纠偏，步伐越来越小，最终收敛到期望值或解。**

这种更新方法在不知道目标函数精确形式、无法获取梯度或概率模型的情形下，提供了一种强大且稳定的近似手段。

---

### 小结对比

| 特性       | Robbins-Monro | 非增量式方法    |
| -------- | ------------- | --------- |
| 是否需要完整数据 | ❌ 只需当前观测      | ✅ 需要多轮样本  |
| 是否适合在线更新 | ✅ 非常适合        | ❌ 不适合     |
| 是否依赖模型结构 | ❌ 不依赖         | 有些方法依赖    |
| 收敛性      | 在一定条件下可收敛     | 取决于算法具体结构 |



## Part 4：随机梯度下降算法介绍（SGD）

---

### 背景与目标

在优化问题中，我们常见如下目标函数：

$$
J(\theta) = \mathbb{E}_{x \sim \mathcal{D}} [\ell(\theta; x)]
$$

其中：

* $\theta$ 是待优化的参数；
* $x \sim \mathcal{D}$ 表示样本来自某个未知分布；
* $\ell(\theta; x)$ 是样本 $x$ 上的损失函数；
* 目标是找到：

  $$
  \theta^* = \arg\min_\theta J(\theta)
  $$

---

### 问题

> 在真实环境中，我们**无法访问所有数据的期望值**，只能从数据分布中采样得到一个数据点 $x_t$，如何更新参数？

---

### 解决方案：随机梯度下降（Stochastic Gradient Descent）

随机梯度下降是 **Robbins-Monro 的推广**，用于求解优化问题中的 **期望最小化问题**。

### 更新公式

每次用一个样本估计梯度，执行参数更新：

$$
\theta_{t+1} = \theta_t - \alpha_t \cdot \nabla_\theta \ell(\theta_t; x_t)
$$

* $x_t$：第 $t$ 次采样样本；
* $\alpha_t$：学习率（step size），通常逐步减小；
* $\nabla_\theta \ell(\theta_t; x_t)$：样本梯度，近似整体目标的梯度。

---

### 与“标准”梯度下降的区别

| 特性   | 批量梯度下降（BGD）   | 随机梯度下降（SGD）     |
| ---- | ------------- | --------------- |
| 梯度计算 | 使用整个训练集       | 使用单一样本估计梯度      |
| 更新频率 | 每个 epoch 更新一次 | 每个样本更新一次        |
| 计算开销 | 大（成本随数据量线性增长） | 小（非常适合在线或大规模数据） |
| 噪声   | 无，梯度精确        | 有噪声，更新路径不稳定     |
| 收敛速度 | 慢（对大数据），但稳定   | 快速迭代，但震荡        |

---

## Part 5：SGD 例子与收敛性

---

### 例子：线性回归中的 SGD

目标函数为：

$$
J(\theta) = \mathbb{E}_{(x,y)} \left[(y - \theta^\top x)^2\right]
$$

SGD 更新公式为：

$$
\theta_{t+1} = \theta_t + \alpha_t (y_t - \theta_t^\top x_t) x_t
$$

每一步都只使用一个样本点 $(x_t, y_t)$ 来逼近整体期望。

---

### SGD 收敛性条件（简述）

SGD 在满足以下条件时可以**收敛到全局最优或局部最优**：

1. **目标函数光滑、梯度 Lipschitz 连续**

2. **学习率满足 Robbins-Monro 条件**：

   $$
   \sum_{t=1}^{\infty} \alpha_t = \infty, \quad \sum_{t=1}^{\infty} \alpha_t^2 < \infty
   $$

   通常设置为：$\alpha_t = \frac{1}{t}$ 或 $\alpha_t = \frac{\alpha_0}{\sqrt{t}}$

3. **梯度噪声的方差有界**

在这些假设下，SGD 具有以下性质：

* 对 **凸函数**，SGD 收敛到全局最优；
* 对 **非凸函数**（如深度学习），SGD 可以收敛到**鞍点或局部最优**；
* 收敛路径不稳定，但平均趋势向最优靠拢。

---

## Part 6：SGD 的有趣性质

---

### 1. 本质是 Robbins-Monro 随机近似

* 每次梯度更新都是对真实梯度的**带噪估计**
* 本质上是**随机根估计法**在高维优化问题中的推广

---

### 2. 噪声有助于跳出鞍点（非凸优化中）

在非凸目标（如神经网络）中，SGD 中的噪声更新：

* 可以**避免卡在鞍点**；
* 有时能逃离不良局部最优；
* 模拟了某种“带温度的优化过程”。

---

### 3. 适合流数据 / 在线学习

* 在 streaming / real-time 系统中，SGD 只需一次采样即可更新参数；
* 不需要完整训练集即可进行持续学习。

---

### 4. 可与 mini-batch 结合（MBGD）

* 实际训练中，常用 **mini-batch SGD**，即用小批量数据估计梯度；
* 兼具稳定性（比纯 SGD 噪声小）与效率（比全量梯度快）：

  $$
  \theta \leftarrow \theta - \alpha \cdot \frac{1}{m} \sum_{i=1}^m \nabla \ell(\theta; x_i)
  $$

---

### 5. 在深度学习中的广泛应用

几乎所有主流深度学习优化器都是 SGD 的变体：

| 方法             | 特性               |
| -------------- | ---------------- |
| SGD            | 基础版本             |
| SGD + momentum | 添加动量项，缓解震荡       |
| RMSProp        | 缩放学习率，缓解梯度爆炸或消失  |
| Adam           | 自适应学习率，结合动量和梯度缩放 |

---

### 6. 有收敛理论支持但需调参

* 收敛性取决于学习率设置；
* 若学习率设置不当（太大或太小），可能发散或极慢；
* 常用做法：**warm-up + decay + early stop**。

---

## 小结

| 属性       | SGD 的表现             |
| -------- | ------------------- |
| 是否增量     | ✅ 是                 |
| 是否适合大规模  | ✅ 非常适合              |
| 是否收敛     | ✅ 若满足条件，可收敛到最优或局部最优 |
| 是否适合非凸问题 | ✅ 适合，能跳出鞍点          |
| 计算效率     | 高，每轮只用一个或一小批样本      |
| 是否用于深度学习 | ✅ 几乎所有模型都基于 SGD 变体  |





## Part 7：SGD vs BGD vs MBGD 比较

---

### 三者定义简要回顾

| 方法                                    | 描述                                  |
| ------------------------------------- | ----------------------------------- |
| **BGD** （Batch Gradient Descent）      | 每次用**所有样本**计算完整梯度后更新参数（适合小数据集）      |
| **SGD** （Stochastic Gradient Descent） | 每次仅使用**一个样本点**来估计梯度（完全增量式）          |
| **MBGD**（Mini-Batch Gradient Descent） | 每次用\*\*一小批样本（如32个）\*\*估计梯度，是两者的折中方式 |

---

### 核心对比表格

| 属性        | BGD        | SGD        | MBGD                |
| --------- | ---------- | ---------- | ------------------- |
| 每次梯度计算样本量 | 全部样本       | 一个样本       | 一小批（如 32/64）        |
| 内存消耗      | 高（需加载全数据）  | 低（只需一个样本）  | 中等                  |
| 每轮计算量     | 高          | 低          | 中等                  |
| 收敛路径      | 平滑、稳定      | 噪声大、抖动     | 相对平稳                |
| 收敛速度      | 慢（尤其是大数据）  | 快（早期）但不稳定  | 收敛较快、表现更好           |
| 并行化难度     | 容易         | 难（串行）      | 容易                  |
| 是否适合大规模数据 | ❌ 不适合      | ✅ 非常适合     | ✅ 非常适合              |
| 是否用于深度学习  | ❌ 较少使用     | ❌ 较少使用     | ✅ 是主流方法             |
| 示例用途      | 理论分析、线性回归等 | 在线学习、流数据任务 | 神经网络训练、DNN、CNN、RNN等 |

---

### 为什么 MBGD 是主流方法？

现代机器学习（特别是深度学习）中，**MBGD 被广泛采用**，原因包括：

* 在硬件上 **可以进行并行加速（如 GPU 向量化）**；
* 噪声足够小，训练过程稳定；
* 能利用 **批量归一化（BatchNorm）** 等技术；
* 支持大规模训练，并容易调节 batch size、learning rate 等超参；
* 有丰富的优化器支持（SGD with momentum、Adam、RMSProp 等）；

---

### 收敛路径可视化（直观图示）

| 梯度下降路径示意图               |
| ----------------------- |
| - BGD：平滑直线下降（但每一步大）     |
| - SGD：路径抖动大但更快尝试多方向     |
| - MBGD：平滑中带有一定扰动，兼顾两者优势 |

---

### 小结结论

| 推荐场景       | 推荐方法         |
| ---------- | ------------ |
| 小数据、精确计算   | BGD          |
| 流式数据、实时更新  | SGD          |
| 大数据、深度模型训练 | ✅ **MBGD** ✅ |

---

### 实用建议

* 通常选择 **MBGD**，batch size 取值一般为 32、64、128；
* 若数据量极大，可结合 **DataLoader + 多线程 + GPU 并行**；
* 若任务是在线学习或带时间结构的，SGD 更加灵活；
* 若任务是理论分析、小样本优化，则 BGD 更易推导。



---

## 课后总结

* **Robbins-Monro** 奠定了在线优化的基础，是 TD、策略优化的理论基石
* **SGD** 是现代深度强化学习的优化核心
* **收敛性**依赖于步长设置和梯度噪声控制
* **在强化学习中，SGD 与策略值函数更新深度融合**



# Q1：什么是 Incremental Algorithms 和 Non-Incremental Algorithms？

---

##  一句话定义：

* **Incremental algorithm（增量式算法）**：每次只处理一个数据点或一个样本，对估计值进行更新，无需存储历史数据。
* **Non-incremental algorithm（非增量式算法）**：依赖**整个数据集或多个样本的累积信息**，通常需要在每次更新中遍历大量（甚至全部）历史数据。

---

##  举例说明

###  Incremental（增量式）方法的例子：

1. **增量均值更新**（Iterative mean）：

   $$
   \mu_n = \mu_{n-1} + \frac{1}{n}(x_n - \mu_{n-1})
   $$

   不需要存储之前所有样本，只需维护一个累计均值。

2. **蒙特卡洛 first-visit 平均更新（Incremental version）**：

   $$
   V(s) \leftarrow V(s) + \alpha [G_t - V(s)]
   $$

   用一步 return $G_t$ 直接更新当前估计值，适用于 streaming 数据。

3. **TD(0) 学习**：

   $$
   V(s) \leftarrow V(s) + \alpha [r + \gamma V(s') - V(s)]
   $$

> ✅ 优点：低存储开销、适合在线学习或流数据环境。

---

### Non-incremental（非增量式）方法的例子：

1. **样本均值（sample average）**：

   $$
   \mu = \frac{1}{n} \sum_{i=1}^{n} x_i
   $$

   需要保存或遍历全部 $n$ 个数据点。

2. **First-visit Monte Carlo**：

   * 对每个状态，记录所有访问回报 $G_1, G_2, ..., G_n$
   * 再取平均更新值函数：

     $$
     V(s) = \frac{1}{n} \sum_{i=1}^{n} G_i
     $$

3. **批量策略评估（Batch Policy Evaluation）**：

   * 收集一批完整 episode 后，一次性计算所有更新。
   * 本质是“延迟更新”。

> 缺点：不适合持续交互环境，内存和计算开销大。

---

## 关键区别总结

| 特性         | Incremental Algorithm | Non-incremental Algorithm |
| ---------- | --------------------- | ------------------------- |
| 是否实时处理     | ✅ 是                   | ❌ 否                       |
| 是否依赖所有历史数据 | ❌ 不依赖                 | ✅ 依赖                      |
| 适合场景       | 在线学习、强化学习、流数据         | 批量学习、离线数据处理               |
| 存储开销       | 小                     | 大                         |
| 计算效率       | 快                     | 慢                         |
| 是否容易并行     | 通常较难                  | 容易（可整体并行）                 |

---

## 在强化学习中的应用对比：

| 算法类型                         | 增量式吗？ | 说明             |
| ---------------------------- | ----- | -------------- |
| Monte Carlo (sample average) | ❌ 否   | 需存储所有回报        |
| Incremental MC               | ✅ 是   | 用步长 α 增量更新     |
| TD(0)                        | ✅ 是   | 增量更新当前状态值      |
| Q-learning                   | ✅ 是   | 增量更新 Q 值       |
| 策略迭代（经典批量）                   | ❌ 否   | 通常在策略评估阶段是非增量的 |




# Q2 为什么优化问题的目标函数可以写成“期望形式”？

---

## 一、背景：机器学习中的优化目标

在机器学习中，我们通常的任务是：

> 在参数空间中找到使模型预测与真实标签尽可能接近的参数值 $\theta$。

比如：

* 在线性回归中，我们最小化均方误差；
* 在分类任务中，我们最小化交叉熵损失；
* 在强化学习中，我们最大化回报的期望。

这些问题的目标函数都可以写为：

$$
\min_\theta \, \mathbb{E}_{x \sim \mathcal{D}} \left[\ell(\theta; x)\right]
$$

---

## 二、原因：我们只接触到了数据的**样本**，而不是**分布**

### 真实目标

我们想最小化的，其实是模型在“所有可能数据点上的平均损失”，即：

$$
\min_\theta \int \ell(\theta; x) \, p(x) \, dx
\quad = \quad
\min_\theta \, \mathbb{E}_{x \sim p(x)} [\ell(\theta; x)]
$$

其中：

* $\ell(\theta; x)$：损失函数；
* $p(x)$：真实数据分布（通常未知）；
* $x$：从真实世界中可能出现的任何输入。

因此我们用期望形式表示，是因为我们的目标是：

> **在整体数据分布上使损失最小化** —— 即最小化“总体期望损失”。

---

### 现实中的做法

由于真实分布 $p(x)$ 不可得，我们只能通过采样（即获取有限数据集）来近似：

$$
\mathbb{E}_{x \sim p(x)} [\ell(\theta; x)] \approx \frac{1}{N} \sum_{i=1}^N \ell(\theta; x_i)
$$

这就是经验风险最小化（Empirical Risk Minimization, ERM）的来源。

---

## 三、强化学习中的例子

强化学习中，目标通常是最大化策略参数下的预期回报：

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} [R(\tau)]
$$

* $\tau$：轨迹；
* $\pi_\theta$：策略定义了在状态下采取动作的概率；
* $R(\tau)$：轨迹获得的总回报。

同样道理，这个目标函数也是期望形式，因为我们要优化的，是**所有可能轨迹下的平均表现**。

---

## 四、总结

| 原因            | 描述                            |
| ------------- | ----------------------------- |
| 真实问题涉及整体分布  | 想要让模型在所有可能数据上表现良好，而不是只对训练样本   |
| 期望形式表达的是总体平均 | 用数学期望来定义“总体的性能”或“总体损失”        |
| 采样估计         | 实际操作中只能通过样本估计期望（SGD 的基础）      |
| 可推广到 RL/NLP 等 | 强化学习、自然语言生成等都以“最大化期望奖励/得分”为目标 |



# Q3 SGD（Stochastic Gradient Descent）与 Robbins-Monro算法的关系、联系与区别

---

## 一、Robbins-Monro算法：起点是一维根估计问题

Robbins-Monro 算法是1951年提出的 **随机近似算法（stochastic approximation）**，解决如下问题：

### 问题定义：

> 设存在某个一维函数 $f(\theta)$，我们希望找到它的“根” $\theta^*$，即：
>
> $$
> f(\theta^*) = \alpha
> $$
>
> 但我们无法直接观察 $f(\theta)$，只能观察带噪的版本：
>
> $$
> Y_t = f(\theta_t) + \text{noise}
> $$

### Robbins-Monro 更新公式：

$$
\theta_{t+1} = \theta_t - \alpha_t (Y_t - \alpha)
$$

其中：

* $\theta_t$：当前估计；
* $\alpha_t$：步长（满足收敛条件）；
* $Y_t$：对 $f(\theta_t)$ 的带噪估计；
* 理解为 “带噪函数值引导参数更新”。

---

## 二、SGD 的定义：在最优化问题中的推广

SGD 的目标是 **最小化期望损失函数**：

$$
J(\theta) = \mathbb{E}_{x \sim \mathcal{D}} [\ell(\theta; x)]
$$

无法直接计算期望时，SGD 用样本近似梯度：

$$
\theta_{t+1} = \theta_t - \alpha_t \nabla_\theta \ell(\theta_t; x_t)
$$

这里：

* $\nabla_\theta \ell(\theta_t; x_t)$ 是对 $\nabla_\theta J(\theta)$ 的无偏估计；
* 更新方向含噪；
* 若 $J(\theta)$ 是凸函数，SGD 可收敛到全局最优。

---

## 三、SGD 是 Robbins-Monro 的自然推广

### 1. Robbins-Monro 是求“标量函数的零点”

* Robbins-Monro 本质上是解决一元方程 $f(\theta) = \alpha$；
* 其迭代形式：

  $$
  \theta_{t+1} = \theta_t - \alpha_t \cdot (f(\theta_t) - \alpha)
  $$
* 可以看作是梯度为 $f(\theta) - \alpha$ 的一维 SGD。

---

### 2. SGD 是在**高维空间上**，以梯度下降方式找极小值点

* SGD 试图使：

  $$
  \nabla_\theta J(\theta) = 0
  $$
* 实际上：

  $$
  \theta_{t+1} = \theta_t - \alpha_t \cdot \widehat{\nabla_\theta J(\theta)}
  $$
* 这就是 Robbins-Monro 方法在**高维 + 多变量** + **梯度形式的函数根估计**上的推广。

---

## 四、二者的数学联系总结

| 项目   | Robbins-Monro                                      | SGD                                        |
| ---- | -------------------------------------------------- | ------------------------------------------ |
| 目标   | 解 $f(\theta) = \alpha$                             | 最小化 $\mathbb{E}[\ell(\theta; x)]$          |
| 方法   | 逐步逼近函数根                                            | 逐步逼近最小值点（梯度为 0）                            |
| 输入   | 带噪观测值 $Y_t = f(\theta_t) + \epsilon_t$             | 带噪梯度估计 $\nabla_\theta \ell(\theta_t; x_t)$ |
| 步长要求 | $\sum \alpha_t = \infty, \sum \alpha_t^2 < \infty$ | 同样适用                                       |
| 本质   | 一维递推随机逼近                                           | 多维梯度随机逼近                                   |

---

## 五、实际意义

* **理论上**：SGD 可以看作是 Robbins-Monro 随机逼近理论的在优化问题上的自然扩展；
* **实践中**：Robbins-Monro 提供了 SGD 学习率调节的**理论依据**；
* **在深度学习中**：即使非凸、非线性问题，SGD 也能通过 Robbins-Monro 理论获得局部收敛性保证。

---

## 小结

| 对比点  | Robbins-Monro           | SGD      |
| ---- | ----------------------- | -------- |
| 起源   | 统计根估计问题                 | 最优化问题    |
| 空间   | 一维（或少数维）                | 高维       |
| 更新方式 | 带噪函数值                   | 带噪梯度     |
| 本质   | 随机近似根解法                 | 随机近似梯度下降 |
| 关系   | SGD 是 Robbins-Monro 的推广 |          |


